# Notebook 09: Progressive Predictions for F1 Fantasy

Make predictions after each session with increasing confidence.

**Normal weekend:** 3 predictions (FP1 → FP2 → FP3)
**Sprint weekend:** 2 predictions (FP1 → Sprint Quali)

**Fantasy deadlines:**
- Normal: Lock lineup after FP3 (before Qualifying)
- Sprint: Lock lineup after Sprint Quali (before Sprint Race)

In [ ]:
import sys
import json
from pathlib import Path

sys.path.append('../')

from src.predictors.team_predictor import rank_teams_for_track

## Load Data

In [ ]:
loaded = []
errors = []

# Load tracks
try:
    track_path = Path('../data/processed/testing_files/track_characteristics/2025_track_characteristics.json')
    with open(track_path) as f:
        track_data = json.load(f)
    all_tracks = track_data.get('tracks', {})
    loaded.append(f"tracks ({len(all_tracks)})")
except FileNotFoundError:
    errors.append("track characteristics")
    all_tracks = {}

# Load cars
try:
    car_path = Path('../data/processed/testing_files/car_characteristics/2025_car_characteristics.json')
    with open(car_path) as f:
        car_data = json.load(f)
    all_cars = car_data.get('teams', {})
    loaded.append(f"teams ({len(all_cars)})")
except FileNotFoundError:
    errors.append("car characteristics")
    all_cars = {}

# Print summary
if loaded:
    print(f"🟢 Loaded: {', '.join(loaded)}")
if errors:
    print(f"🔴 Missing: {', '.join(errors)}")

# Show available sessions for first team (if exists)
if all_cars:
    first_team = list(all_cars.keys())[0]
    print(f"\nExample ({first_team}): {len(all_cars[first_team])} sessions")

## Demo 1: Normal Weekend - Bahrain

**Timeline:**
- Friday: FP1 → FP2
- Saturday morning: FP3 ← **LOCK FANTASY LINEUP!**
- Saturday afternoon: Qualifying (too late)

Watch confidence increase: 0.20 → 0.35 → 0.42

In [ ]:
track_name = 'Bahrain Grand Prix'
bahrain = all_tracks.get(track_name)

if bahrain:
    print(f"Progressive Predictions: {track_name}")
    print("=" * 80)
    
    # Make 3 predictions
    sessions = [
        ('post_fp1', 'Post-FP1 (Friday afternoon)'),
        ('post_fp2', 'Post-FP2 (Friday evening)'),
        ('post_fp3', 'Post-FP3 (Saturday morning)')
    ]
    
    for session_key, label in sessions:
        
        rankings = rank_teams_for_track(
            all_cars, 
            bahrain, 
            session_key, 
            'normal'
            )

        session_conf = rankings[0][2]

        print(f"\n{label} — Conf: {session_conf:.2f}")
        if rankings:
            print(f"   {rankings[0][3]}")
        print("-" * 80)
        
        for rank, (team, score, conf, reason) in enumerate(rankings, 1):
            print(f"{rank}. {team:<25} Score: {score:.3f}")
        
else:
    print(f"{track_name} not found in track data")

## Demo 2: Sprint Weekend - Miami (if available)

**Timeline:**
- Friday: FP1 (60 min) → Sprint Quali ← **LOCK FANTASY LINEUP!**
- Saturday: Sprint Race (too late)

Only 2 predictions, lower confidence (less practice time).

In [ ]:
# Try to find a sprint weekend (Miami, Austria, USA, Brazil, Qatar)
sprint_tracks = ['Miami Grand Prix', 'Austrian Grand Prix', 'United States Grand Prix', 
                 'São Paulo Grand Prix', 'Qatar Grand Prix']

sprint_track = None
for track in sprint_tracks:
    if track in all_tracks:
        sprint_track = track
        break

if sprint_track:
    sprint_chars = all_tracks[sprint_track]
    
    print(f"Progressive Predictions: {sprint_track} (Sprint)")
    print("=" * 80)
    
    sessions = [
        ('post_fp1', 'Post-FP1 (Friday) - Only 60 minutes!'),
        ('post_sprint_quali', 'Post-Sprint Quali (Friday) ← LOCK LINEUP!')
    ]
    
    for session_key, label in sessions:

        rankings = rank_teams_for_track(all_cars, sprint_chars, session_key, 'sprint')

        session_conf = rankings[0][2]

        print(f"\n{label} — Conf: {session_conf:.2f}")
        if rankings:
            print(f"   {rankings[0][3]}")
        print("-" * 80)
        
        for rank, (team, score, conf, reason) in enumerate(rankings[:5], 1):
            print(f"{rank}. {team:<25} Score: {score:.3f}")
else:
    print("No sprint weekend tracks found in data")
    print("(Need to extract Miami, Austria, USA, Brazil, or Qatar)")

## Demo 3: Track Comparison - Monaco vs Monza

**Different tracks favor different cars!**

- Monaco: Tight, twisty → Good at corners
- Monza: High-speed → Fast on straights

In [ ]:
monaco = all_tracks.get('Monaco Grand Prix')
monza = all_tracks.get('Italian Grand Prix')

if monaco and monza:
    print("Track Comparison (Post-FP3):")
    print("=" * 80)
    
    print("\nMonaco (tight, twisty):")
    print("-" * 80)
    monaco_rankings = rank_teams_for_track(all_cars, monaco, 'post_fp3', 'normal')
    for rank, (team, score, conf, _) in enumerate(monaco_rankings[:5], 1):
        print(f"{rank}. {team:<25} {score:.3f}")
    
    print("\nMonza (high-speed):")
    print("-" * 80)
    monza_rankings = rank_teams_for_track(all_cars, monza, 'post_fp3', 'normal')
    for rank, (team, score, conf, _) in enumerate(monza_rankings[:5], 1):
        print(f"{rank}. {team:<25} {score:.3f}")
    
    print("\n→ Rankings should be different!")
    print("   Teams good at corners (Monaco) ≠ Teams fast on straights (Monza)")
else:
    print("Monaco or Monza not found")

## Demo 4: Check Data Quality

Diagnose if rankings look wrong.

In [ ]:
from src.features.normalization import extract_all_teams_performance

print("Data Quality Check:")
print("=" * 80)

# Extract performance from FP1
perf = extract_all_teams_performance(all_cars, 'fp1')

if perf:
    print(f"🟢 Extracted {len(perf)} teams\n")
    
    # Show performance spread
    teams_sorted = sorted(perf.items(), key=lambda x: x[1].get('slow_corner_performance', 0), reverse=True)
    
    print("Corner Performance (sorted):")
    print("-" * 80)
    for team, p in teams_sorted[:5]:
        corners = p.get('slow_corner_performance', 0)
        speed = p.get('top_speed', 0)
        print(f"{team:<25} Corners: {corners:.3f}  Speed: {speed:.3f}")
    
    print("\nIf all scores are similar (0.4-0.6), FP1 data is not representative.")
    print("Teams sandbag in FP1. Need FP2/FP3 for realistic rankings.")
else:
    print("🔴 Extraction failed!")
    print("Check that JSON has 'sector_times' and 'speed_profile'")